In [ ]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/processed-ontonotes5.json

In [2]:
import json
from sklearn.model_selection import train_test_split
import random

In [3]:
with open('processed-ontonotes5.json') as fopen:
    data = json.load(fopen)

In [4]:
from collections import defaultdict

entities = defaultdict(list)
for i in data:
    entities['text'].append(i[0])
    entities['label'].append(i[1])

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/magiceventstartup.20180920.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/magiceventstartup.20191216.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/jabatan.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/maklumat-syarikat-jualan-langsung.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/penganjur-acara.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/persatuan.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/ORG/senarai-anak-syarikat-kplb.csv

In [6]:
import pandas as pd

df = pd.read_csv('penganjur-acara.csv')['Nama Syarikat'].dropna()
local_companies = df.tolist()

In [7]:
df = pd.read_csv('senarai-anak-syarikat-kplb.csv')['NAMA SYARIKAT'].dropna()
local_companies.extend(df.tolist())

In [8]:
df = pd.read_csv('magiceventstartup.20191216.csv')['Startup Name'].dropna()
local_companies.extend(df.tolist())

In [9]:
df = pd.read_csv('magiceventstartup.20180920.csv')['Startup Name'].dropna()
local_companies.extend(df.tolist())

In [10]:
df = pd.read_csv('maklumat-syarikat-jualan-langsung.csv')['NAMA SYARIKAT'].dropna()
local_companies.extend(df.tolist())

len(local_companies)

4633

In [11]:
import re

local_companies = [re.sub(r'[ ]+', ' ', l.replace('.', ' ')).strip().title() for l in local_companies]
local_companies = list(set(local_companies))
len(local_companies)

3147

In [12]:
org = local_companies

In [13]:
with open('jabatan.txt') as fopen:
    data = fopen.read().split('\n')
    
jabatan = [i.strip() for i in data if len(i) > 2]
len(jabatan)

426

In [14]:
with open('persatuan.txt') as fopen:
    data = fopen.read().split('\n')
    
persatuan = [i.strip() for i in data if len(i) > 2]
len(persatuan)

224

In [17]:
big_it_companies = """
Apple
Microsoft
Google
Tencent
Facebook
IBM
SAP
Instagram
Accenture
Intel
Adobe
Samsung
Salesforce
LinkedIn
Huawei
Oracle
Cisco
Dell
Xiaomi
Baidu
AWS
GCP
Sony
"""
big_it_companies = list(filter(None, big_it_companies.split('\n')))

In [18]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'ORG':
        while entities['label'][i] == 'ORG':
            r.append(i)
            i += 1
        results.append(r)
    i += 1
    
len(results)

24612

In [39]:
import math
import numpy as np

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [20]:
train_results, test_results = train_test_split(results, test_size = 0.2)
train_org, test_org = train_test_split(org, test_size = 0.2)

In [33]:
train_X, train_Y = [], []

repeat = 10
for t in train_org:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

25170

In [34]:
for t in big_it_companies:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
        
len(train_X)

25400

In [35]:
repeat = 15
for t in jabatan:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

31790

In [36]:
repeat = 15
for t in persatuan:
    for i in range(repeat):
        x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)
            
len(train_X)

35150

In [40]:
test_X, test_Y = [], []
repeat = 10

for t in test_org:
    for i in range(repeat):
        x, y = generate_index(test_results[random.randint(0, len(test_results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        test_X.append(x)
        test_Y.append(y)
            
len(test_X)

6300

In [41]:
len(train_X), len(test_X)

(35150, 6300)

In [ ]:
with open('augmentation-org-ontonotes5.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y,
              'test_X': test_X, 'test_Y': test_Y}, fopen)